# Using the Commodity Futures and Options data-set

The python version of this sample can be found [here](https://github.com/shyams80/plutons/blob/master/docs-py/CommodityFuturesAndOptions.ipynb)

In [1]:
library(dbplyr)
library(dplyr)
library(odbc)
library(plutoR)
options("scipen"=999)
source("config.R")

#initialize
commodityFO <- CommodityFuturesAndOptions()


Attaching package: ‘dplyr’

The following objects are masked from ‘package:dbplyr’:

    ident, sql

The following objects are masked from ‘package:stats’:

    filter, lag

The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union



### get all the commodities that are being traded in COMEX and NYMEX

In [2]:
maxDt <- (commodityFO$CmeEod() %>%
          summarize(MAX_DT = max(TIME_STAMP)) %>%
          collect())$MAX_DT[[1]]

t1 <- commodityFO$CmeEod() %>%
  filter(TIME_STAMP == maxDt) %>%
  group_by(PRODUCT_SYMBOL, PRODUCT_DESCRIPTION) %>%
  summarize(TOTAL_VOLUME = sum(VOLUME)) %>%
  select(PRODUCT_SYMBOL, PRODUCT_DESCRIPTION, TOTAL_VOLUME) %>%
  collect()

t2 <- commodityFO$CmeEod() %>%
  group_by(PRODUCT_SYMBOL) %>%
  summarize(START_DT = min(TIME_STAMP)) %>%
  select(PRODUCT_SYMBOL, START_DT) %>%
  collect()

t1 %>%
  inner_join(t2, by=c('PRODUCT_SYMBOL')) %>%
  filter(TOTAL_VOLUME > 0) %>%
  arrange(desc(TOTAL_VOLUME)) %>%
  print()

Warning message:
“Missing values are always removed in SQL.
Use `MAX(x, na.rm = TRUE)` to silence this warning
This warning is displayed only once per session.”Warning message:
“Missing values are always removed in SQL.
Use `SUM(x, na.rm = TRUE)` to silence this warning
This warning is displayed only once per session.”Warning message:
“Missing values are always removed in SQL.
Use `MIN(x, na.rm = TRUE)` to silence this warning
This warning is displayed only once per session.”

# A tibble: 119 x 4
# Groups:   PRODUCT_SYMBOL [119]
   PRODUCT_SYMBOL PRODUCT_DESCRIPTION                    TOTAL_VOLUME START_DT  
   <chr>          <chr>                                         <dbl> <date>    
 1 CL             Light Sweet Crude Oil Futures               1098386 2013-11-25
 2 GC             Gold Futures                                 565590 2013-11-25
 3 NG             Henry Hub Natural Gas Futures                328816 2013-11-25
 4 SI             Silver Futures                               182041 2013-11-25
 5 HG             Copper Futures                               160775 2013-11-25
 6 RB             Rbob Gasoline Futures                        155504 2013-11-25
 7 HO             Ny Harbor Ulsd Futures                       142055 2013-11-25
 8 BZ             Brent Crude Oil Last Day Financial Fu…        99092 2013-11-25
 9 CLT            Crude Oil Tas Futures                         50149 2013-11-25
10 MGC            E-micro Gold Futures                  

### get all the commodity futures that are being traded in MCX

In [3]:
maxDt <- (commodityFO$McxEod() %>%
            summarize(MAX_DT = max(TIME_STAMP)) %>%
            collect())$MAX_DT[[1]]

t1 <- commodityFO$McxEod() %>%
  filter(TIME_STAMP == maxDt & (OTYPE == 'XX' | OTYPE == 'FUTCOM')) %>%
  group_by(CONTRACT) %>%
  summarize(TOTAL_OI = sum(OI)) %>%
  select(CONTRACT, TOTAL_OI) %>%
  collect()

t2 <- commodityFO$McxEod() %>%
  group_by(CONTRACT) %>%
  summarize(START_DT = min(TIME_STAMP)) %>%
  select(CONTRACT, START_DT) %>%
  collect()

t1 %>%
  inner_join(t2, by=c('CONTRACT')) %>%
  filter(TOTAL_OI > 0) %>%
  arrange(desc(TOTAL_OI)) %>%
  print()

# A tibble: 22 x 3
   CONTRACT   TOTAL_OI START_DT  
   <chr>         <dbl> <date>    
 1 CRUDEOILM     43780 2015-01-06
 2 SILVERMIC     36278 2011-02-18
 3 GOLD          30740 2003-11-10
 4 GOLDM         23030 2003-11-20
 5 SILVER        18557 2003-11-10
 6 NATURALGAS    17641 2006-07-11
 7 GOLDPETAL     16666 2011-04-18
 8 NICKEL        15391 2004-06-03
 9 CRUDEOIL      14686 2005-02-09
10 SILVERM       14598 2004-02-17
# … with 12 more rows


### get all the commodity futures that are being traded in NCDEX

In [4]:
maxDt <- (commodityFO$NcdexEod() %>%
            summarize(MAX_DT = max(TIME_STAMP)) %>%
            collect())$MAX_DT[[1]]

t1 <- commodityFO$NcdexEod() %>%
  filter(TIME_STAMP == maxDt) %>%
  group_by(COMMODITY) %>%
  summarize(TOTAL_OI = sum(OI)) %>%
  select(COMMODITY, TOTAL_OI) %>%
  collect()

t2 <- commodityFO$NcdexEod() %>%
  group_by(COMMODITY) %>%
  summarize(START_DT = min(TIME_STAMP)) %>%
  select(COMMODITY, START_DT) %>%
  collect()

t1 %>%
  inner_join(t2, by=c('COMMODITY')) %>%
  filter(TOTAL_OI > 0) %>%
  arrange(desc(TOTAL_OI)) %>%
  print()


# A tibble: 13 x 3
   COMMODITY              TOTAL_OI START_DT  
   <chr>                     <int> <date>    
 1 CASTORSEEDNEW            246595 2009-09-03
 2 GUAR_SEED10              120550 2013-09-05
 3 RAPE_MUSTARD_SEEDS        99820 2010-11-10
 4 COTTONSEEDOILCAKEAKOLA    97780 2009-09-03
 5 GUARGUM_5MT               66805 2016-05-02
 6 SOY_OIL                   63830 2015-06-01
 7 CORIANDER                 33780 2008-08-11
 8 WHEATKOTA                 10780 2017-12-01
 9 MAIZERABI                  3300 2016-02-26
10 KAPASNEW                   2087 2018-06-14
11 BARLEY                     1800 2006-12-11
12 MOONG                       735 2019-07-08
13 PADDY_BASMATI                80 2019-07-10


This notebook was created using [pluto](http://pluto.studio). Learn more [here](https://github.com/shyams80/pluto)